In [3]:
#网页模式抓取二手房成交记录
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

OriginURL = r"https://sh.lianjia.com/chengjiao/"
Header ={
"Host": "sh.lianjia.com",
"Connection": "keep-alive",
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
"Accept": "image/webp,image/apng,image/*,*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "zh-CN,zh;q=0.9"}


def get_block_list(url):
    _connection = requests.get(url)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _BlockListOrigin = _BS.find("div", class_="position").find_all("a", href=re.compile("/chengjiao/\w+/"), rel=False)
    _BlockList = ["https://sh.lianjia.com" + i["href"] for i in _BlockListOrigin]
    return _BlockList


def get_block_list_of_a_city(url):
    _BlockList = []
    _connection = requests.get(url)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _DistrictListOrigin = _BS.find_all("a", href=re.compile("/chengjiao/\w+/"), title=re.compile("上海"))
    _DistrictList = ["https://sh.lianjia.com" + i["href"] for i in _DistrictListOrigin]
    print("Fetched the {}".format(url))
    for i in _DistrictList:
        templist = get_block_list(i)
        _BlockList = _BlockList + templist
        print("Fetched the {}".format(i))
    _BlockList = set(_BlockList)-set(_DistrictList)
    return _BlockList


def save_file(data, filename, path=os.getcwd(), filetype="txt"):
    """

    :param data:
    :param filename:
    :param path:
    :param filetype:
    :return:
    """
    if filetype == 'txt':
        with open("{0}/{1}".format(path, filename), "w") as f:
            for i in data:
                f.write("{}\n".format(i))
                
                
save_file(get_block_list_of_a_city(OriginURL),"BlockList.txt")

Fetched the https://sh.lianjia.com/chengjiao/
Fetched the https://sh.lianjia.com/chengjiao/pudong/
Fetched the https://sh.lianjia.com/chengjiao/minhang/
Fetched the https://sh.lianjia.com/chengjiao/baoshan/
Fetched the https://sh.lianjia.com/chengjiao/xuhui/
Fetched the https://sh.lianjia.com/chengjiao/putuo/
Fetched the https://sh.lianjia.com/chengjiao/yangpu/
Fetched the https://sh.lianjia.com/chengjiao/changning/
Fetched the https://sh.lianjia.com/chengjiao/songjiang/
Fetched the https://sh.lianjia.com/chengjiao/jiading/
Fetched the https://sh.lianjia.com/chengjiao/huangpu/
Fetched the https://sh.lianjia.com/chengjiao/jingan/
Fetched the https://sh.lianjia.com/chengjiao/zhabei/
Fetched the https://sh.lianjia.com/chengjiao/hongkou/
Fetched the https://sh.lianjia.com/chengjiao/qingpu/
Fetched the https://sh.lianjia.com/chengjiao/fengxian/
Fetched the https://sh.lianjia.com/chengjiao/jinshan/
Fetched the https://sh.lianjia.com/chengjiao/chongming/
Fetched the https://sh.lianjia.com/che

In [69]:
#API方式地图抓取
import json
import time
import hashlib
import requests

group_type = {
    "district":0.3,     #行政区
    "bizcircle":0.08,    #商圈
    "community":0.01     #居民小区
}
b= {"city_id": "310000",
     "group_type": "bizcircle",
     "max_lat": round(31.21516171001514, 6),
     "min_lat": round(31.21516171001514, 6) - 0.08,
     "max_lng": round(121.49389948569473, 6),
     "min_lng": round(121.49389948569473, 6) - 0.08,
   }

def get_md5(txt):
    """md5加密函数"""
    m = hashlib.md5()
    m.update(txt.encode('utf-8'))
    return m.hexdigest()

def get_line_site(url):
    """请求链接"""
    headers = {'Accept': '*/*',
               'Accept-Encoding': 'gzip, deflate, br',
               'Accept-Language': 'zh-CN,zh;q=0.9',
               'Connection': 'keep-alive',
               'Host': 'ajax.lianjia.com',
               'Referer': 'https://gz.lianjia.com/ditu/',
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 '
                             '(KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}
    r = requests.get(url, headers=headers)
    return r

def get_url(bbb):
    """拼接链接"""
    url ="https://ajax.lianjia.com/map/search/ershoufang/?city_id={city_id}&group_type={group_type}&" \
         "max_lat={max_lat}&min_lat={min_lat}&max_lng={max_lng}&min_lng={min_lng}&" \
         "filters=%7B%7D&request_ts={request_ts}&source=ljpc&" \
         "authorization={authorization}"
    request_ts = int(time.time() * 1000)
    md5_data = "vfkpbin1ix2rb88gfjebs0f60cbvhedlcity_id={city_id}group_type={group_type}max_lat={max_lat}" \
             "max_lng={max_lng}min_lat={min_lat}min_lng={min_lng}request_ts={request_ts}".format(
        city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"],
        request_ts=request_ts)
    authorization = get_md5(md5_data)
    url = url.format(request_ts=request_ts, authorization=authorization,city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"])
    return url

def getres(bb):
    line_url = get_url(bb)
    print(line_url)
    res = get_line_site(line_url)
    items = res.json()['data']
    return items 

getres(b)


https://ajax.lianjia.com/map/search/ershoufang/?city_id=310000&group_type=bizcircle&max_lat=31.215162&min_lat=31.135162&max_lng=121.493899&min_lng=121.413899&filters=%7B%7D&request_ts=1537259964696&source=ljpc&authorization=d267d6d27cf478c708a1400b03b9edeb


In [4]:
#获取城市纬度
import json

ak = "dASz7ubuSpHidP1oQWKuAK3q"
def getCor(city):
    url = "http://api.map.baidu.com/geocoder/v2/?address={city}&output=json&ak={ak}".format(city=city,ak=ak)
    res = requests.get(url).json()["result"]["location"]
    return res

def getCode(city):
    Cor = getCor(city)
    url2 = "http://api.map.baidu.com/geocoder/v2/?location={lat},{lng}&output=json&pois=1&ak={ak}".format(lat=Cor["lat"], lng=Cor["lng"],ak=ak)
    print(url2)
    res = requests.get(url2).json()
    return res

getCor("北京")
    

{'lng': 116.39564503787867, 'lat': 39.92998577808024}

In [51]:
#生成城市列表
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

res = requests.get("https://sh.lianjia.com/city/")
work = BeautifulSoup(res.text, "lxml")
res = set([(i["href"], i.string) for i in work.find_all("a",href=re.compile(r"(?<=https://)\w+(?=\.lianjia.com/)"),target=False)][:-1])
citycodemapping = pd.read_csv(r"C:\Users\wenpeiyu\Desktop\citycodemapping.csv")
dfcitylist = pd.DataFrame({
    "city": [i[1] for i in res],
    "city_url": [i[0] for i in res]})
dfCityList = pd.merge(dfcitylist,citycodemapping,how="inner",on=["city"])
lng = [getCor(i+"市")['lng'] for i in dfCityList.city]
lat = [getCor(i+"市")['lat'] for i in dfCityList.city]
minlng = [i-0.3 for i in lng]
minlat = [i-0.3 for i in lat]
city = [i for i in dfCityList.city]
dfCityCor = pd.DataFrame({
    "city": city,
    "max_lat": lat,
    "max_lng": lng,
    "min_lat": minlat,
    "min_lng": minlng
})
dfCityList = pd.merge(dfCityList,dfCityCor,how="inner",on=["city"])
dfCityList = dfCityList.rename(columns={"citycode":"city_id"})
dfCityList.to_csv(r".\etc\CityList.csv", index=False)

In [61]:
round(dfCityList,6)

In [62]:
tempdict ={}
for i in dfCityList.index:
    tempdict[dfCityList.loc[i,"city"]]=dict(dfCityList.loc[i,["city_url","city_id","max_lat","max_lng","min_lat","min_lng"]])

In [65]:
"上海" in tempdict

True

In [17]:
#encoding=UTF-8
import requests
import base64

def base_code(username, password):
    str = '%s:%s' % (username, password)
    encodestr = base64.b64encode(str.encode('utf-8'))
    return '%s' % encodestr.decode()

def q():
    usernaem = "dxxxhly@qq.com"  # 您的用户名
    password = "455190881Lx"  # 您的密码
    proxy_ip = "120.27.9.96" # 代理ip，通过http://h.wandouip.com/get获得
    proxy_port = "80"  # 代理端口号
    head = {
        'Proxy-Authorization': 'Basic %s' % (base_code(usernaem, password))
    }
    proxy = {
        'http': 'http://%s:%s' % (proxy_ip, proxy_port),
        'https': 'https://%s:%s' % (proxy_ip, proxy_port)
    }
    url = 'http://myip.ipip.net'
    try:
        text = requests.get(url,proxies=proxy,timeout=5,headers=head)#, proxies=proxy ,timeout=5)
        print(text.text)
    except Exception as e:
        print("Error: ", e)
while True:
    q()

Error:  HTTPConnectionPool(host='120.27.9.96', port=80): Read timed out. (read timeout=5)


KeyboardInterrupt: 

In [55]:
import requests
def verification(proxy_ip,proxy_port):
    header = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "h-CN,zh;q=0.9",
    "connection":"keep-alive",
    "Host": "myip.ipip.net",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"
        }
    proxy = {
        'http': 'http://%s:%s' % (proxy_ip, proxy_port),
        'https': 'https://%s:%s' % (proxy_ip, proxy_port)
    }
    url = 'http://myip.ipip.net'
    try:
        text = requests.get(url,headers=header, proxies=proxy,timeout=5)
        print(text.text)
        if len(text.text)<100:
            return (proxy_ip,proxy_port)
    except Exception as e:
        print("Error: ", e)


Error:  HTTPConnectionPool(host='183.129.207.74', port=14823): Read timed out. (read timeout=5)


In [56]:
with open("e:/wenpeiyu/proxy.txt", "r") as f:
    a = f.readlines()

for i in set(a):
    eval("verification"+i[:-1])

Error:  HTTPConnectionPool(host='123.207.237.42', port=3128): Max retries exceeded with url: http://myip.ipip.net/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000000007AE7A58>, 'Connection to 123.207.237.42 timed out. (connect timeout=5)'))
当前 IP：118.190.95.35  来自于：中国 山东 青岛  阿里云/电信/联通/移动/铁通/教育网

当前 IP：222.76.204.110  来自于：中国 福建 厦门  电信

当前 IP：111.72.155.7  来自于：中国 江西 宜春  电信

Error:  HTTPConnectionPool(host='116.1.11.19', port=80): Max retries exceeded with url: http://myip.ipip.net/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000000056624A8>, 'Connection to 116.1.11.19 timed out. (connect timeout=5)'))
Error:  HTTPConnectionPool(host='58.51.83.102', port=808): Max retries exceeded with url: http://myip.ipip.net/ (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))
当前 IP：211.159.171.58  来自于：中国 北京 北京  电信/联通/移动

<h1>Not Found</h1>The requested URL http://m

In [67]:
verification('123.207.237.42', '3128')

当前 IP：123.207.237.42  来自于：中国 广东 广州  电信/联通/移动



('123.207.237.42', '3128')